<a href="https://colab.research.google.com/github/AliKarimiENT/NLP_HW1/blob/main/NLPHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [ ]:
pip install hazm

Imports

In [ ]:
import re
from os import remove
from hazm import word_tokenize
from hazm import Stemmer
from tensorflow import keras
from keras.preprocessing import text
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import re
import nltk
import sys
from collections import OrderedDict
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

Reading txt file and getting verses

Here we need to do some processes

In [ ]:
# with this function we check that it contains number in the text or not
def hasNumbers(inputString) :
  for character in inputString :
    if character.isdigit() :
      return True
  return False

def hasDash(inputString) :
  for character in inputString :
    if character == '-' :
      return True
  return False 


with open('/content/ShamsDaftar6.doc_filename=ShamsDaftar6.txt') as f:
  dataSet = f.readlines()

verses = []
for line in dataSet :
  if line != ' ' and line != '\n' and hasNumbers(line) == False  and hasDash(line)==False:
    if dataSet.index(line) != 0 :
      verses.append(line.replace('\t',' '))


# for i in range(20):
#   print(verses[i])


In [ ]:
with open('/content/stopwords.txt') as f:
  stops = f.readlines()
stopWords = []
for stop in stops :
  stopWords.append(re.sub('\n','',stop.replace('\ufeff','')))
print(stopWords)

Now it's time to use **Hazm** **Library** to tokenize each sentence

In [ ]:
concept_words = []
for verse in verses:
  # now we must add all words in the list to words list
  # for word in word_tokenize(verse):
  for word in verse.split():
    concept_words.append(word)

In [ ]:
print(concept_words)

Remove stop words from verses

In [ ]:
# for removing stop words from our words list we need to this process
corpus  = [] #concept words list is a list that contains list of words without stopWords
stemmer = Stemmer()
for word in concept_words :
  wd = stemmer.stem(word)
  if word not in stopWords:
    if wd not in stopWords:
        corpus.append(wd)
    # else:
    #     if word != wd :
    #       corpus.append(word)
words = []
for word in corpus:
  words.append(word)
corpus = set(corpus)
print(corpus)

In [ ]:
print(len(corpus))



**Tokenize** corpus vocabulary

Import keras and use it 

Now,we want a list of our sentences as list of words:

In [ ]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus)

# wordToIndex = tokenizer.word_index
# indexToWord = {v:k for k,v in wordToIndex.items()}
# print(wordToIndex)
wordToIndex = {}
indxToWord = {}


vocab_size = len(corpus)
for i,word in enumerate(corpus):
    wordToIndex[word] = i
    indxToWord[i] = word 
embed_size = 100

# print(corpus)
# windows = [[wordToIndex[w] for w in text.text_to_word_sequence(doc)] for doc in corpus]
print(vocab_size)
print(wordToIndex.items())
# Now each unique word from the corpus is a part of our vocabulary now with a unique numeric identifier



In [ ]:
sentences = []
for verse in verses:
  sentence = []
  for word in verse.split():
    wd = stemmer.stem(word)
    if word not in stopWords:
      if wd not in stopWords:
        if wd not in sentence:
          sentence.append(wd) 
      # else:
      #   if word != wd :
      #     if word not in sentence :
      #       sentence.append(word)
        # sentence.append(stemmer.stem(word))
  sentences.append(sentence)
  
print(sentences)

Now we will train our data

In [ ]:
from tensorflow.python.data.ops.dataset_ops import AUTOTUNE
data = []
WINDOW_SIZE = 2
for sentence in sentences :
  for word_index , word in enumerate(sentence) :
    for nb_word in sentence[max(word_index - WINDOW_SIZE,0) : min(word_index + WINDOW_SIZE , len(sentences)) + 1] :
      if nb_word != word :
        data.append([word,nb_word])
# dataset = tf.data.T
data = np.asarray(data)
tensor1 = tensorflow.data.Dataset.from_tensor_slices(data)
print(type(tensor1))
tensor1 = tensor1.cache().prefetch(buffer_size=AUTOTUNE)
print(tensor1)


Lets define our **One Hot Function**

This function will convert numbers to one hot vectors

In [ ]:
def to_one_hot(data_point_index,vocab_size):
  temp = np.zeros(vocab_size)
  temp[data_point_index] = 1
  return temp

In [ ]:
print(len(data))

Initail x and y train values 

x train is a list of our input words in one hot form

y train is a list of out output words in one hot form


In [ ]:
# print(tensor1.as_numpy_iterator())
# list(tensor1.as_numpy_iterator())

In [ ]:
from tensorflow.python.keras import backend 

x_train = []
y_train = []


for data_word in data:
  
  x_train.append(tf.one_hot(wordToIndex[data_word[0]],vocab_size))
  y_train.append(tf.one_hot(wordToIndex[data_word[1]],vocab_size))

#convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

print(x_train)

Both x_traing and y_train have shape :

In [ ]:
print(x_train.shape,y_train.shape)

Now it's time to make **Tensorflow model**


In [ ]:
tf.compat.v1.disable_eager_execution()

x = tf.compat.v1.placeholder(tf.float32, shape=(None, vocab_size),name=None)
y_label = tf.compat.v1.placeholder(tf.float32, shape=(None, vocab_size),name=None)

print(x)
print(y_label)

Convert training data into **Embedded representation**

In [ ]:
EMBEDDING_DIM = 5
W1 = tf.Variable(tf.random.normal([vocab_size,EMBEDDING_DIM]))
b1 = tf.Variable(tf.random.normal([EMBEDDING_DIM])) #bias

hidden_representation = tf.add(tf.matmul(x,W1),b1)

print(W1)
print(b1)
print(hidden_representation)

Making a prediction about neighbours with softmax

In [ ]:
W2 = tf.Variable(tf.random.normal([EMBEDDING_DIM,vocab_size]))
b2 = tf.Variable(tf.random.normal([vocab_size]))

prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_representation,W2),b2))
print(prediction)